## Step 1: Set up variables 

1. Set up a environment variable that points to the secret key that was shared with everyone. We get this API key using the GCP console.
2. Also set up constant values.

In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'secret.json'

In [2]:
import dialogflow_v2 as dialogflow
PROJECT_ID = 'poc1-b624d'
SESSION_ID = '12-ab'
import pandas as pd
# CONTEXT_NAME_INPUT = 'input_context'
# CONTEXT_NAME_OUTPUT = 'output_context'
# CONTEXT_NAME_INPUT = "projects/{}/agent/sessions/{}/contexts/{}".format(PROJECT_ID,SESSION_ID, CONTEXT_NAME_INPUT)
# CONTEXT_NAME_OUTPUT = "projects/{}/agent/sessions/{}/contexts/{}".format(PROJECT_ID,SESSION_ID, CONTEXT_NAME_OUTPUT) 
# LIFESPAN = 5

In [3]:
?dialogflow.types.Intent

## Step 2: Entity creation

In this section we will create an entity. We will create the entity value as well as synonyms used for different values in the entity. 

In [4]:
## This cell contains the code for entity creation.

def create_entity_type(project_id, display_name, kind):
    """Create an entity type with the given display name."""
    entity_types_client = dialogflow.EntityTypesClient()
    parent = entity_types_client.project_agent_path(project_id)
    entity_type = dialogflow.types.EntityType(
        display_name=display_name, 
        kind=kind
    )
    response = entity_types_client.create_entity_type(parent, entity_type)
    print('Entity type created: \n{}'.format(response))
    return
    
    
def list_entity_types(project_id):
    """
    This is a helper function that lists down all entities in existence
    """
    entity_types_client = dialogflow.EntityTypesClient()
    parent = entity_types_client.project_agent_path(project_id)
    entity_types = entity_types_client.list_entity_types(parent)
    #
    for entity_type in entity_types:
        print('Entity type name: {}'.format(entity_type.name))
        print('Entity type display name: {}'.format(entity_type.display_name))
        print('Number of entities: {}\n'.format(len(entity_type.entities)))
    #
    return
        
        
def _get_entity_type_ids(project_id, display_name):
    """
    Helper to get entity_type_id from display name.
    """
    entity_types_client = dialogflow.EntityTypesClient()
    #
    parent = entity_types_client.project_agent_path(project_id)
    entity_types = entity_types_client.list_entity_types(parent)
    entity_type_names = [ entity_type.name for entity_type in entity_types if entity_type.display_name == display_name]
    entity_type_ids = [ entity_type_name.split('/')[-1] for entity_type_name in entity_type_names]
    #
    return entity_type_ids


def create_entity(project_id, entity_type_id, entity_value, synonyms):
    """
    Create an entity of the given entity type.
    UPDATE : 
        take entity value as list
        take synonymns as list of list
    """
    import dialogflow_v2 as dialogflow
    entity_types_client = dialogflow.EntityTypesClient()

    # Note: synonyms must be exactly [entity_value] if the
    # entity_type's kind is KIND_LIST
    
    synonyms = synonyms or [entity_value]

    entity_type_path = entity_types_client.entity_type_path(
        project_id, entity_type_id)
    #
    entity_arr = []
    for i in range(len(entity_value)) : 
        # Create entity    
        entity = dialogflow.types.EntityType.Entity()
        entity.value = entity_value[i]
        entity.synonyms.extend(synonyms[i])
        entity_arr += [entity]
    #
    response = entity_types_client.batch_create_entities(
        entity_type_path, 
        entity_arr
    )

    print('Entity created: {}'.format(response))
    return

In [5]:

entities_file = "entity_input_v3.xlsx"
l = pd.ExcelFile(entities_file)
entity_names_list = l.sheet_names
#
for sname in entity_names_list : 
    # Create the main entity
    create_entity_type(project_id = PROJECT_ID, 
               display_name = '{}'.format(sname ), 
               kind = 'KIND_MAP')

    #
    m = pd.read_excel(entities_file, sheet_name= sname, header=None,index_col=0)
    m.columns = list(range(m.shape[1]))
    m = m.fillna("")
    m['comb'] = m.apply(lambda x : ",".join(x), axis=1 )
    # Create dict
    d = dict(zip([ str(x) for x in list(m.index)], [ str(x )for x in list(m.comb)] ))
    # Create Sub entities
    k_arr = []
    synn_arr = []
    for k in d.keys() : 
        #
        k_arr += [k]
        synn_arr += [d[k].split(",")]
    print(k_arr)
    print(synn_arr)
    #
    create_entity(project_id = PROJECT_ID, 
              entity_type_id = _get_entity_type_ids(
                  project_id = PROJECT_ID, 
                  display_name = '{}'.format(sname)
              )[0], 
              entity_value = k_arr, 
              synonyms =  synn_arr
    )

FailedPrecondition: 400 EntityType with display_name 'PAYER' already exists.

## Step 3: Basic intent creation

This will create an intent through API. Parameters/entities in the train set will be identified through a custom data input data style. 

In [14]:

def create_intent_using_file(excel_file_name):
    """
    This will take in a csv file. The output will be that the intents will be created.
    Sentences need not be broken down.
    """
    # Read sheetnames
    l = pd.ExcelFile(excel_file_name)
    intents_name_list = l.sheet_names
    #
    for intent in intents_name_list : 
        # Loop over each file
        m = pd.read_excel(
            excel_file_name, 
            sheet_name= intent, 
            header=None).reset_index()
        m.columns = ['id', 'data']
        m['data'] = m['data'].apply(lambda x : [{"text" : x}])
        tr = {
            "userSays" : m.to_dict(orient='records')
        }
        # tr is the structure in input format
        # Now train the structure
        print(create_intent_v3(project_id = PROJECT_ID, 
                         display_name = intent, 
                         training_phrases_dict = tr, 
                         message_texts=[]))
    return



def create_intent_v3(project_id, display_name, training_phrases_dict, message_texts):
    """
    Create an intent of the given intent type.
    
    This takes in a custom data type set. 
    """
    import dialogflow_v2 as dialogflow
    intents_client = dialogflow.IntentsClient()

    parent = intents_client.project_agent_path(project_id)
    training_phrases = []
    gg = []
    
    # Convert the custom data into MORE usable data
    for x in training_phrases_dict['userSays'] : 
        gg += [x['data']]

        
    # Create the training phrases here
    for training_phrases_part in gg:
        parts_array = []
        
        for tp in training_phrases_part :
            if len(tp.keys())  > 1 : 
                # This is the special part of the sentence
                # add the other 4 parts
                parts_array += [dialogflow.types.Intent.TrainingPhrase.Part(
                    text=tp['text'], 
                    entity_type = tp['meta'],
                    alias = tp['alias'] ,
                    user_defined = tp['userDefined'])
                ]
            else:
                # This is the single part of the sentence
                parts_array += [dialogflow.types.Intent.TrainingPhrase.Part(text=tp['text'])]
        
        # Here we create a new training phrase for each provided part.
        training_phrase = dialogflow.types.Intent.TrainingPhrase(parts=parts_array, type=2)
        training_phrases.append(training_phrase)

    # Create default messages for intent to be used as a response.
    text = dialogflow.types.Intent.Message.Text(text=message_texts)
    message = dialogflow.types.Intent.Message(text=text)

    # Actual intent command 
    intent = dialogflow.types.Intent(
        display_name=display_name,
        training_phrases=training_phrases,
        messages=[message])

    # We get the response message after the intent has been created
    response = intents_client.create_intent(parent, intent)

    print('Intent created: {}'.format(response))

In [17]:
# create_intent_v3(project_id= PROJECT_ID, 
#               display_name = 'SALES8', 
#               training_phrases_dict = tp2,
#               message_texts = ['Sales detected'])
create_intent_using_file("intent_creation_v2.xlsx")

Intent created: name: "projects/poc1-b624d/agent/intents/fb263aae-0d65-4ba4-931b-a72c7a69c24a"
display_name: "sales_template"
priority: 500000
messages {
  text {
  }
}

None


In [25]:
from dialogflow_v2 import intents_client
name = "projects/poc1-b624d/agent/intents/fb263aae-0d65-4ba4-931b-a72c7a69c24a"
myintent = intents_client.IntentsClient().get_intent(name,intent_view=intents_client.enums.IntentView.INTENT_VIEW_FULL)
myintent.Parameter.mandatory = True
lang='en'
response = intents_client.IntentsClient().update_intent(myintent, lang)

In [31]:
myintent

name: "projects/poc1-b624d/agent/intents/fb263aae-0d65-4ba4-931b-a72c7a69c24a"
display_name: "sales_template"
priority: 500000
training_phrases {
  name: "613d3cc9-faff-496a-bafc-7fe80c05ab95"
  type: TEMPLATE
  parts {
    text: "what are my sales for @HCP:HCP ?"
  }
}
training_phrases {
  name: "699fb545-23af-495b-8c6a-ce4eb99f0604"
  type: TEMPLATE
  parts {
    text: "what are @HCP:HCP \'s sales for @PRODUCT:PRODUCT"
  }
}
messages {
  text {
  }
}

In [23]:
myintent.Parameter.mandatory

name: "projects/poc1-b624d/agent/intents/fb263aae-0d65-4ba4-931b-a72c7a69c24a"
display_name: "sales_template"
priority: 500000
training_phrases {
  name: "613d3cc9-faff-496a-bafc-7fe80c05ab95"
  type: TEMPLATE
  parts {
    text: "what are my sales for @HCP:HCP ?"
  }
}
training_phrases {
  name: "699fb545-23af-495b-8c6a-ce4eb99f0604"
  type: TEMPLATE
  parts {
    text: "what are @HCP:HCP \'s sales for @PRODUCT:PRODUCT"
  }
}
messages {
  text {
  }
}

## Step 4 : Detect intent through API

The aim of this is to access an existing bot through the API. We will display the results.

In [41]:
def detect_intent_texts(project_id, session_id, texts, language_code):
    """
    
    Returns the result of detect intent with texts as inputs. 
    Using the same `session_id` between requests allows continuation of the conversaion.
    
    """
    import dialogflow_v2 as dialogflow
    session_client = dialogflow.SessionsClient()
    # Create session path
    session = session_client.session_path(project_id, session_id)
    
    ans = []
    for text in texts:
        # Save the response for input query here
        rsp = {}
        
        # The process to get output
        text_input = dialogflow.types.TextInput(text=text, language_code=language_code)
        query_input = dialogflow.types.QueryInput(text=text_input)
        response = session_client.detect_intent(session=session, query_input=query_input)
        
        # Create the output response
        rsp['input_query'] = text # Input question
        rsp['intent'] = {
            'detected_intent' : "{}".format(response.query_result.intent.display_name),
            'intent_confidence' : "{}".format(round(response.query_result.intent_detection_confidence, 4))
        } 
        d = {}
        for k in list(response.query_result.parameters) : 
            if k == 'fields' : 
                pass
            else:
                val = response.query_result.parameters[k]
                #print(k , val)
                if len(val) == 1 : 
                    d[k] = val[0]
                else : 
                    d[k] = val
        rsp['parameters'] = d
        
        # Adding that to the array of input questions
        ans += [rsp]        
    return ans

In [47]:
# Set the questions
ques =  ['What are sales by Dr Bruce Morrison for lectrazine']

# The actual function call
er = detect_intent_texts(project_id = PROJECT_ID, 
                    session_id = SESSION_ID, 
                    texts =ques, 
                    language_code= 'en')
# Print the results
for i, k in enumerate(er) :
    print("------------------{}---------------".format(i))
    print(k)

------------------0---------------
{'input_query': 'What are sales by Dr Bruce Morrison for lectrazine', 'intent': {'detected_intent': 'SALES8', 'intent_confidence': '0.41'}, 'parameters': {'DATE': '', 'PRODUCT': 'lectrazine ', 'METRIC': , 'HCP': 'Bruce Morrison'}}


A known issue is 

https://github.com/googleapis/dialogflow-python-client-v2/issues/25

We still need to find a workaround for this.